# OSACT 6 Task 2: Translating Arabic Dialects to Modern Standard Arabic

Among the challenges involved with using Arabic in large language models is the disparity of dialects used throughout the Arab world. While modern Standard Arabic is used prolifically in official and government use-cases, among the general public, it is seldom ever used. Instead, various **vernaculars** are used, local to various regions across the Middle East and North Africa. These vernaculars can be divided into subgroups, the count of which can vary from the single digits to dozens, depending on the level of granularity through which one defines the geographic differences.

The dataset provided by OSACT-6 aggregates the dialects as follows:
    
- Egyptian
- Iraqi
- Levantine
- Maghrebi
- Gulf

The dataset contains 200 sentences from each dialect, with its corresponding translation in MSA. As such, the baseline naive model accuracy the machine learning model should be expected to beat is 0.2.

## EDA

First, we load the dataset and show some sample data.

In [42]:
import pandas as pd

dataset = pd.read_json("./dev_set_all.json")
dataset = dataset.set_index("id")
dataset.sample(5)

,dialect,source,target
id,,,
418603,Egyptian,طيب طيب يا ماما,حسنًا حسنًا يا أمي
422449,Magharebi,الله يسلمك أحلام، ربي يحفظك ختيتو,الله يحفظك أحلام، ربي يحميك أختي
420210,Levantine,قيمتها في الواقع أقل من القيمة التي ينتجها ويخ...,قيمتها الفعلية أقل من القيمة التي يتم إنتاجها ...
422189,Magharebi,تقودي نتي تلعبي مع دراري وتفاهات نتاعك يااا .....,"""كلمة نابية"" أنتِ وتلعبين مع الصغار وتفاهاتك ي..."
420214,Levantine,هلا بس تحكي أنه في شبه مشكلة بالضرائب بتبلش تف...,عندما تقول أن هناك مشكلة تقريبية في الضرائب، ت...


In [55]:
print(len(dataset["target"].unique()))

1001


We can get a better understanding of how the model should perform by examining the differences between dialects and MSA. Each MSA target is unique, suggesting that each source-target pair is unique in meaning. This complicates the analysis, but it should be doable given the appropriate approach.

ValueError: No axis named Egyptian for object type DataFrame